In [44]:
import random
import numpy as np
import numpy.random as npRand
import scipy.stats as stats

In [45]:
starData = np.loadtxt("Astro Lab Star Data.txt")
starData = np.transpose(starData)
B, dB = starData[0], starData[1]
D, dD = starData[2], starData[3]

In [46]:
#calculating theoretical dR
dR = 1.6 *(dB**2 + .02**2 + 1.6 *(dD**2 + .02**2))**0.5

In [47]:
#choosing random data sets for each star type
def getData(mean, sigma, numVals):
    data = np.array(
        [npRand.normal(m, s, numVals) for m, s in zip(mean, sigma)]
    )
    return data

In [54]:
B_vals = getData(B, dB, 1000)
D_vals = getData(D, dD, 1000)

In [55]:
A = 1 + B
C = 1/1.6 + D
dA = dC = np.linspace(0.02, 0.02, 50)
A_vals = getData(A, dA, 1000)
C_vals = getData(C, dC, 1000)

In [56]:
#calculating the "Monte Carlo" values for R and dR
R_vals = (A_vals - B_vals) / (C_vals - D_vals)
R_mc = np.mean(R_vals, axis=1)
dR_mc = np.std(R_vals, axis=1)

In [57]:
#Next I need to test "Goodness of fit"
numBins = 20
counts, Bins = zip(*np.stack(
    np.histogram(r_vals, numBins) for r_vals in R_vals
))
    

#calc expected with stats cdf and np.diff
exp = np.diff(np.stack(
    stats.norm.cdf(bins, r_mc, dr_mc) for bins, r_mc, dr_mc in zip(Bins, R_mc, dR_mc)
))

#scale expected percentages by sample size
exp = 1000 * exp

#Now calculate the Chi^2 values:
chi2 = np.sum(( (counts - exp)**2 / exp), axis=1)

In [58]:
chi2/numBins

array([2.93779970e+00, 8.02451690e+00, 2.70574709e+00, 2.04639596e+00,
       1.00273578e+02, 1.15917103e+01, 3.47739000e+00, 2.95159416e+00,
       1.90359109e+00, 5.01579344e+00, 3.06089460e+00, 8.30147423e+00,
       1.61663433e+00, 5.69235804e+01, 3.37583327e+00, 2.60224684e+00,
       2.56231675e+00, 4.46956366e+00, 1.05856528e+00, 3.02828467e+00,
       1.61243200e+00, 4.35591471e+00, 1.30163951e+01, 1.61668572e+01,
       1.56755765e+00, 5.17430921e+00, 1.94539079e+00, 2.57433880e+00,
       2.32948213e+00, 2.19254866e+00, 2.92690647e+00, 4.43498311e+01,
       1.71393151e+03, 4.41512754e+00, 1.30092531e+00, 3.22117075e+00,
       1.22771168e+02, 4.29671141e+00, 3.09892498e+00, 2.92597244e+00,
       2.49155282e+00, 2.60382971e+00, 4.66739007e+00, 1.93089091e+00,
       2.83965713e+00, 5.55649210e+00, 9.42096003e-01, 1.51968616e+00,
       1.71126750e+00, 5.97986464e+00])